In [ ]:
!pip install keras-tuner
!pip install openpyxl
!pip install tensorflow
!pip install optuna
!pip install optuna-integration[tfkeras]
!pip install flask pyngrok tensorflow scikit-learn joblib --quiet
!pip install flask-ngrok
!pip install pyngrok

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras_tuner as kt
import io
import shutil
import joblib
import random
import optuna
import random
import threading
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.losses import mse
from tensorflow.keras.metrics import mae
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from google.colab import files
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from optuna.integration import TFKerasPruningCallback
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU, Reshape, Lambda, Multiply, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from tensorflow.keras.models import load_model
from flask import Flask, request, jsonify
from pyngrok import ngrok

In [ ]:
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_excel(io.BytesIO(uploaded[file_name]))

Saving eksdidi (3).xlsx to eksdidi (3).xlsx


In [ ]:
# Reset tuner directory
shutil.rmtree('tuner_results/crop_yield_bayesian', ignore_errors=True)

# Fix random seeds
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# Load Dataset
label_encoder = LabelEncoder()
df['Soil Type'] = label_encoder.fit_transform(df['Soil Type'])

selected_columns = ['Soil Type', 'Moisture_1', 'Moisture_2', 'Moisture_3',
                    'Luminosity_1', 'Luminosity_2', 'Luminosity_3',
                    'Duration_1', 'Duration_2', 'Duration_3', 'Total Duration', 'Yield/Plant']
df = df[selected_columns]

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Upsampling
def upsample_high_yield(X, y, threshold=8.0, factor=3):
    high_idx = np.where(y >= threshold)[0]
    X_high = X[high_idx]
    y_high = y[high_idx]
    X_upsampled = np.concatenate([X] + [X_high] * (factor - 1), axis=0)
    y_upsampled = np.concatenate([y] + [y_high] * (factor - 1), axis=0)
    indices = np.arange(len(X_upsampled))
    np.random.shuffle(indices)
    return X_upsampled[indices], y_upsampled[indices]

X_upsampled, y_upsampled = upsample_high_yield(X, y, threshold=8.0, factor=3)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_upsampled, y_upsampled, test_size=0.2, random_state=42)

# Padding
num_features_needed = 12

def pad_dataset(data, num_features_needed, pad_value=0.0):
    if data.shape[1] < num_features_needed:
        padding = np.full((data.shape[0], num_features_needed - data.shape[1]), pad_value)
        data = np.concatenate([data, padding], axis=1)
    return data

X_train = pad_dataset(X_train, num_features_needed)
X_test = pad_dataset(X_test, num_features_needed)

# Scaling
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape for CNN
X_train = X_train.reshape(X_train.shape[0], 3, 4, 1)
X_test = X_test.reshape(X_test.shape[0], 3, 4, 1)

# Weighted MSE
def weighted_mse(y_true, y_pred):
    weight = 1.0 + tf.square((y_true - 8.0) / 2.0)
    error = tf.square(y_true - y_pred)
    return tf.reduce_mean(weight * error)

def softmax_axis1(z):
    return K.softmax(z, axis=1)

def softmax_output_shape(shape):
    return shape

def sum_axis1(z):
    return K.sum(z, axis=1)

def sum_output_shape(shape):
    return (shape[0], shape[2])

# CNN Model
def create_model(trial):
    inputs = Input(shape=(3, 4, 1))
    x = inputs
    for i in range(5):
        x = Conv2D(filters=trial.suggest_int(f'conv{i+1}_filters', 32, 128, step=32),
                   kernel_size=trial.suggest_categorical(f'conv{i+1}_kernel', [2, 3]),
                   padding='same', kernel_regularizer=l2(1e-4))(x)
        x = LeakyReLU(0.1)(x)
        x = BatchNormalization()(x)
        x = Dropout(0.2)(x)

    flatten_shape = x.shape[1] * x.shape[2]
    channels = x.shape[-1]
    reshaped = Reshape((flatten_shape, channels))(x)

    attention_scores = Dense(1, activation='tanh')(reshaped)
    attention_weights = Lambda(softmax_axis1, output_shape=softmax_output_shape)(attention_scores)
    context_vector = Multiply()([reshaped, attention_weights])
    context_vector = Lambda(sum_axis1, output_shape=sum_output_shape)(context_vector)

    x = Dense(trial.suggest_int('dense_units', 64, 128, step=32), kernel_regularizer=l2(1e-4))(context_vector)
    x = LeakyReLU(0.1)(x)
    x = Dropout(0.2)(x)

    output = Dense(1, activation='linear')(x)
    model = Model(inputs, output)

    lr = trial.suggest_categorical('learning_rate', [1e-3, 5e-4, 1e-4])
    model.compile(optimizer=Adam(learning_rate=lr), loss=weighted_mse, metrics=['mae'])
    return model

# Optuna Objective
def objective(trial):
    model = create_model(trial)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=30,
        batch_size=16,
        verbose=0,
        callbacks=[early_stopping, reduce_lr, TFKerasPruningCallback(trial, 'val_loss')]
    )
    return min(history.history['val_mae'])

# Run Optuna
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=2, timeout=1200)
print("Best Trial:")
print(study.best_trial.params)

# Train Final CNN
best_model = create_model(study.best_trial)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)

history = best_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=16,
    callbacks=[early_stopping, reduce_lr]
)

# Save Model
best_model.save("best_crop_yield_model_optuna.keras")

# Random Forest + Ensemble
cnn_preds = best_model.predict(X_test).flatten()
X_train_rf = X_train.reshape(X_train.shape[0], -1)
X_test_rf = X_test.reshape(X_test.shape[0], -1)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_rf, y_train)
rf_preds = rf_model.predict(X_test_rf)

stacked_input = np.vstack((cnn_preds, rf_preds)).T

# Stacked Ensemble - Gradient Boosting
meta_model_gb = GradientBoostingRegressor(n_estimators=600, learning_rate=0.1, max_depth=3, random_state=42)
meta_model_gb.fit(stacked_input, y_test)
gb_preds = meta_model_gb.predict(stacked_input)

[I 2025-06-17 02:16:56,005] A new study created in memory with name: no-name-9272386d-ceb6-43c3-8561-aec8fd0d1925
[I 2025-06-17 02:21:30,028] Trial 0 finished with value: 1.1551836729049683 and parameters: {'conv1_filters': 64, 'conv1_kernel': 2, 'conv2_filters': 96, 'conv2_kernel': 2, 'conv3_filters': 32, 'conv3_kernel': 2, 'conv4_filters': 96, 'conv4_kernel': 3, 'conv5_filters': 128, 'conv5_kernel': 2, 'dense_units': 64, 'learning_rate': 0.0005}. Best is trial 0 with value: 1.1551836729049683.
[I 2025-06-17 02:29:23,397] Trial 1 finished with value: 1.1301639080047607 and parameters: {'conv1_filters': 64, 'conv1_kernel': 2, 'conv2_filters': 64, 'conv2_kernel': 3, 'conv3_filters': 128, 'conv3_kernel': 3, 'conv4_filters': 96, 'conv4_kernel': 3, 'conv5_filters': 32, 'conv5_kernel': 3, 'dense_units': 128, 'learning_rate': 0.001}. Best is trial 1 with value: 1.1301639080047607.


Best Trial:
{'conv1_filters': 64, 'conv1_kernel': 2, 'conv2_filters': 64, 'conv2_kernel': 3, 'conv3_filters': 128, 'conv3_kernel': 3, 'conv4_filters': 96, 'conv4_kernel': 3, 'conv5_filters': 32, 'conv5_kernel': 3, 'dense_units': 128, 'learning_rate': 0.001}
Epoch 1/50
840/840 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - loss: 22.4299 - mae: 2.6250 - val_loss: 10.8782 - val_mae: 1.8671 - learning_rate: 0.0010
Epoch 2/50
840/840 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - loss: 11.3983 - mae: 1.7205 - val_loss: 8.0224 - val_mae: 1.2644 - learning_rate: 0.0010
Epoch 3/50
840/840 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - loss: 10.7056 - mae: 1.6419 - val_loss: 7.5107 - val_mae: 1.3203 - learning_rate: 0.0010
Epoch 4/50
840/840 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - loss: 9.9844 - mae: 1.5942 - val_loss: 7.0725 - val_mae: 1.3018 - learning_rate: 0.0010
Epoch 5/50
840/840 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - loss: 9.5746 - mae: 1.5454 - val_loss: 7.4495 - val_mae: 1.3557 - learning_rate: 0.0010
Epoch 6/50
840/840

In [ ]:
# Save Gradient Boosting Meta Model
joblib.dump(meta_model_gb, "final_meta_model_gb.pkl")

# Save Scaler
joblib.dump(scaler, "scaler.pkl")

# Save Label Encoder (if you’ll use it later for new data)
joblib.dump(label_encoder, "soil_label_encoder.pkl")

#Save Random Fores
joblib.dump(rf_model, "rf_model.pkl")

['rf_model.pkl']

In [ ]:
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from keras.config import enable_unsafe_deserialization
from sklearn.preprocessing import LabelEncoder

enable_unsafe_deserialization()

# === Custom Loss ===
def weighted_mse(y_true, y_pred):
    weight = 1.0 + tf.square((y_true - 8.0) / 2.0)
    error = tf.square(y_true - y_pred)
    return tf.reduce_mean(weight * error)

# === Load Models & Tools ===
best_model = load_model(
    "best_crop_yield_model_optuna.keras",
    custom_objects={
        'weighted_mse': weighted_mse,
        'softmax_axis1': softmax_axis1,
        'softmax_output_shape': softmax_output_shape,
        'sum_axis1': sum_axis1,
        'sum_output_shape': sum_output_shape
    }
)
rf_model = joblib.load("rf_model.pkl")
meta_model_gb = joblib.load("final_meta_model_gb.pkl")
scaler = joblib.load("scaler.pkl")
label_encoder = joblib.load("soil_label_encoder.pkl")

# === Predict Function ===
def predict_crop_yield(input_dict):
    soil_type_encoded = input_dict['Soil Type']

    features = [
        soil_type_encoded,
        input_dict['Moisture_1'],
        input_dict['Moisture_2'],
        input_dict['Moisture_3'],
        input_dict['Luminosity_1'],
        input_dict['Luminosity_2'],
        input_dict['Luminosity_3'],
        input_dict['Duration_1'],
        input_dict['Duration_2'],
        input_dict['Duration_3'],
        input_dict['Total Duration']
    ]
    features = np.array(features, dtype=np.float32).reshape(1, -1)

    # Pad to 12 features if needed
    if features.shape[1] < 12:
        features = np.concatenate([features, np.zeros((1, 12 - features.shape[1]), dtype=np.float32)], axis=1)

    # Scale features
    features_scaled = scaler.transform(features)

    # CNN prediction: ensure shape (1, 3, 4, 1) and dtype float32
    features_cnn = tf.convert_to_tensor(features_scaled.reshape(1, 3, 4, 1), dtype=tf.float32)
    cnn_pred = best_model.predict(features_cnn, verbose=0).flatten()

    # RF prediction
    rf_pred = rf_model.predict(features_scaled.reshape(1, -1))

    # Stacked ensemble prediction
    stacked_input = np.vstack((cnn_pred, rf_pred)).T
    final_pred = meta_model_gb.predict(stacked_input)

    return final_pred[0]

# === Example Usage ===
new_sample = {
    'Soil Type': 0,
    'Luminosity_1': 5847.85,
    'Luminosity_2': 4339.5,
    'Luminosity_3': 1489.4,
    'Moisture_1': 90.61,
    'Moisture_2': 41.56,
    'Moisture_3': 81.77,
    'Duration_1': 36,
    'Duration_2': 34,
    'Duration_3': 38,
    'Total Duration': 108
}

predicted_yield = predict_crop_yield(new_sample)
print(f"Predicted Yield/Plant: {predicted_yield:.2f}")

Predicted Yield/Plant: 5.59


In [ ]:
soil_types = ['Loamy', 'Sandy']

label_encoder = LabelEncoder()
label_encoder.fit(soil_types)

joblib.dump(label_encoder, "soil_label_encoder.pkl")

print("LabelEncoder classes:", label_encoder.classes_)


best_model = tf.keras.models.load_model(
    "best_crop_yield_model_optuna.keras",
    custom_objects={
        'weighted_mse': weighted_mse,
        'softmax_axis1': softmax_axis1,
        'softmax_output_shape': softmax_output_shape,
        'sum_axis1': sum_axis1,
        'sum_output_shape': sum_output_shape
    }
)
rf_model = joblib.load("rf_model.pkl")
meta_model_gb = joblib.load("final_meta_model_gb.pkl")
scaler = joblib.load("scaler.pkl")
label_encoder = joblib.load("soil_label_encoder.pkl")

def weighted_mse(y_true, y_pred):
    weight = 1.0 + tf.square((y_true - 8.0) / 2.0)
    error = tf.square(y_true - y_pred)
    return tf.reduce_mean(weight * error)

def predict_crop_yield(input_dict):
    # Ensure SoilType is a plain Python string
    soil_type_encoded = label_encoder.transform([str(input_dict['SoilType'])])[0]

    features = [
        soil_type_encoded,
        input_dict['Moisture_1'],
        input_dict['Moisture_2'],
        input_dict['Moisture_3'],
        input_dict['Luminosity_1'],
        input_dict['Luminosity_2'],
        input_dict['Luminosity_3'],
        input_dict['Duration_1'],
        input_dict['Duration_2'],
        input_dict['Duration_3'],
        input_dict['Total Duration']
    ]
    features = np.array(features).reshape(1, -1)

    if features.shape[1] < 12:
        features = np.concatenate([features, np.zeros((1, 12 - features.shape[1]))], axis=1)

    # Scale features
    features_scaled = scaler.transform(features)

    # CNN prediction
    features_cnn = features_scaled.reshape(1, 3, 4, 1)
    cnn_pred = best_model.predict(features_cnn).flatten()

    # RF prediction
    features_rf = features_scaled.reshape(1, -1)
    rf_pred = rf_model.predict(features_rf)

    # Stacked ensemble prediction
    stacked_input = np.vstack((cnn_pred, rf_pred)).T
    final_pred = meta_model_gb.predict(stacked_input)

    return final_pred[0]

# === Flask API setup ===
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()  # Get JSON data from the request
        if not data:
            return jsonify({'error': 'No data received'}), 400
        print(f"Received data: {data}")  # Log received data for debugging
        prediction = predict_crop_yield(data)
        return jsonify({'predicted_yield': prediction}), 200
    except Exception as e:
        print(f"Error: {e}")  # Log the exception for debugging
        return jsonify({'error': str(e)}), 400  # Handle any errors

# Function to start the Flask server
def start_flask():
    app.run(host='0.0.0.0', port=5000)

# Function to start Ngrok tunnel
def start_ngrok():
    public_url = ngrok.connect(5000)
    print(f'Ngrok tunnel "{public_url}" is running')

# Create separate threads for Flask and Ngrok
flask_thread = threading.Thread(target=start_flask)
ngrok_thread = threading.Thread(target=start_ngrok)

# Start both threads
flask_thread.start()
ngrok_thread.start()

LabelEncoder classes: ['Loamy' 'Sandy']
 * Serving Flask app '__main__'
 * Debug mode: off


In [ ]:
!ngrok authtoken 123456789 #replace this with your own ngrok authorization token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!ngrok http 5000

INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 03:01:27] "GET /robots.txt HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 03:01:28] "GET /predict HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 03:01:32] "GET /robots.txt HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 03:01:32] "GET /robots.txt HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 03:01:32] "GET /predict HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 03:01:33] "GET /predict HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 03:01:33] "GET /robots.txt HTTP/1.1" 404 -


Received data: {'SoilType': 'Loamy', 'Luminosity_1': 4535.0, 'Luminosity_2': 4024.0, 'Luminosity_3': 4754.0, 'Moisture_1': 65.38981, 'Moisture_2': 66.61212, 'Moisture_3': 72.7235947, 'Duration_1': 27.13377, 'Duration_2': 29.08944, 'Duration_3': 29.08944, 'Total Duration': 85.31265}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 03:02:40] "POST /predict HTTP/1.1" 200 -


Received data: {'SoilType': 'Sandy', 'Luminosity_1': 2345.0, 'Luminosity_2': 4299.0, 'Luminosity_3': 3235.0, 'Moisture_1': 62.9452171, 'Moisture_2': 83.5695953, 'Moisture_3': 83.68281, 'Duration_1': 29.08944, 'Duration_2': 26.5685959, 'Duration_3': 23.0017929, 'Total Duration': 78.65983}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 03:04:06] "POST /predict HTTP/1.1" 200 -


...

INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 03:25:57] "POST /predict HTTP/1.1" 200 -


Received data: {'SoilType': 'Sandy', 'Luminosity_1': 3113.0, 'Luminosity_2': 4504.0, 'Luminosity_3': 4463.0, 'Moisture_1': 82.31096, 'Moisture_2': 82.19778, 'Moisture_3': 78.76819, 'Duration_1': 28.4891987, 'Duration_2': 29.3123055, 'Duration_3': 27.1173363, 'Total Duration': 84.91884}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 03:34:22] "POST /predict HTTP/1.1" 200 -


Received data: {'SoilType': 'Loamy', 'Luminosity_1': 4791.0, 'Luminosity_2': 3276.0, 'Luminosity_3': 3809.0, 'Moisture_1': 56.93179, 'Moisture_2': 75.45178, 'Moisture_3': 82.99694, 'Duration_1': 31.55254, 'Duration_2': 34.02188, 'Duration_3': 26.3395157, 'Total Duration': 91.91394}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 03:37:58] "POST /predict HTTP/1.1" 200 -
